In [1]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
max_seq_length = 2048 # Supports RoPE Scaling interally, so choose any!

/drive0-storage/aryaduta_ta/my-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.2.1+cu121)
    Python  3.10.14 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [2]:
alpaca_prompt = """Di bawah ini adalah instruksi yang menggambarkan sebuah tugas dengan input yang memberikan konteks lebih lanjut. Tuliskan respons yang tepat untuk menyelesaikan permintaan tersebut.

### Instruksi:
{}

### Input:
{}

### Response:
{}"""

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Aryaduta/modellora9",
    max_seq_length = max_seq_length,
    dtype = None,
)


/drive0-storage/aryaduta_ta/my-env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Gemma patching release 2024.4
   \\   /|    GPU: NVIDIA A100-SXM4-40GB MIG 7g.40gb. Max memory: 39.392 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Unsloth 2024.4 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


In [4]:
full_instruction = """
Objektif: Tugas anda adalah menghasilkan urutan respons JSON untuk merencanakan tindakan lengan robot berdasarkan input pengguna. Jika tujuan tidak dapat dicapai dengan menggunakan instruksi yang disediakan dan objek yang tersedia, kembalikan pesan kesalahan.
    
Berikan objek JSON yang mengandung array "actions", diidentifikasi dengan key "actions".
    
Setiap aksi harus direpresentasikan sebagai objek dengan "command" dan "parameters" yang sesuai
    
Objek dan Koordinat yang Tersedia:
{object_context}
    
Perintah yang Tersedia:
1. move: Gerakkan lengan robot ke arah tertentu. Sertakan parameter "direction" dengan nilai "atas", "bawah", "depan", "belakang", "kiri", atau "kanan".
2. move_to: Gerakkan lengan robot ke koordinat tertentu. Sertakan parameter "x", "y", dan "z" untuk menentukan koordinat tujuan.
3. suction_cup: Aktifkan atau nonaktifkan cup hisap. Gunakan parameter "action" dengan nilai "on" atau "off".
5. err_msg: Kembalikan pesan kesalahan jika tujuan pengguna tidak dapat tercapai dengan menggunakan objek dan perintah saat ini. Gunakan parameter "msg" dengan nilai "tidak dapat membuat rencana aksi dengan kondisi terkini".
    
Contoh Penggunaan Perintah:
{{"actions":[{{"command":"move","parameters":{{"direction":"up"}}}},{{"command":"move_to","parameters":{{"x":-265.56,"y":196.34,"z":-50}}}},{{"command":"suction_cup","parameters":{{"action":"on"}}}},{{"command":"err_msg","parameters":{{"msg":"tidak dapat membuat rencana aksi dengan kondisi terkini"}}}}]}}
    
Instruksi Penggunaan:
1. Untuk memindahkan objek yang tersedia ke koordinat tertentu, aktifkan penyedot terlebih dahulu menggunakan perintah "suction_cup" dengan "action" diatur ke "on", kemudian gerakkan ke koordinat objek menggunakan perintah "move_to".
2. Berikan koordinat penempatan untuk tujuan pengguna menggunakan perintah "move_to" sesuai dengan koordinat yang diberikan pengguna atau koordinat objek yang disediakan. Jangan lupa untuk menambahkan tanda negati
3. Untuk melepaskan objek setelah menggunakan penyedot, nonaktifkan penyedot terlebih dahulu menggunakan perintah "suction_cup" dengan "action" diatur ke "off".
4. Untuk memindahkan robot secara lateral (misalnya, ke kiri atau kanan), gunakan perintah "move" dengan arah yang sesuai.
5. Untuk gerakan lateral objek (kiri atau kanan), pertama-tama gerakkan lengan robot ke koordinat objek menggunakan perintah "move_to", kemudian gunakan perintah "move" dengan arah yang sesuai.
6. Jika tujuan pengguna tidak dapat tercapai dengan perintah dan objek saat ini, gunakan perintah "err_msg".
"""

def inferrence(input_context, object_context):
    # Enable native 2x faster inference
    FastLanguageModel.for_inference(model)

    full_instruction2 = full_instruction.format(object_context=object_context)

    
    # Prepare inputs
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                full_instruction2,
            input_context,  # input
            "",  # output - leave this blank for generation!
        )
        ],
        return_tensors="pt"
    ).to("cuda")
    
    # Generate outputs
    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
    
    # Decode output and extract JSON response
    decoded_output = tokenizer.batch_decode(outputs)[0]
    
    # Find the start and end indices for the JSON response
    start_marker = "### Response:"
    end_marker = "<eos>"
    start_index = decoded_output.find(start_marker) + len(start_marker)
    end_index = decoded_output.find(end_marker, start_index)
    
    # Extract the JSON response
    json_response = decoded_output[start_index:end_index].strip()
    
    # Print the JSON response
    print(json_response)

    # return the JSON response
    return json_response


In [5]:
import getpass
import os
import threading

from flask import Flask, request, jsonify
from pyngrok import ngrok, conf

conf.get_default().auth_token = "XXXX"

app = Flask(__name__)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url
# Define Flask routes
@app.route("/")
def index():
    return "Hello from Jupyter DGX-A-100"

@app.route('/api')
def api():
    data = request.json
    input_context = data.get('input_context', None)
    object_context = data.get('object_context', None)
    
    if input_context and object_context:
        output_data = inferrence(input_context, object_context)
        return jsonify(output_data)
    elif input_context is None and object_context is None:
        return jsonify({"status": "failure", "message": "input_context and object_context are missing"}), 400
    else:
        return jsonify({"status": "failure", "message": "input_context or object_context is missing"}), 400


# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "https://05c0-103-94-190-27.ngrok-free.app" -> "http://127.0.0.1:5000/"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jul/2024 18:48:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2024 18:48:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Jul/2024 18:49:12] "GET /api HTTP/1.1" 200 -


{"actions": [{"command": "move_to", "parameters": {"x": 12, "y": 13, "z": 15}}, {"command": "suction_cup", "parameters": {"action": "on"}}, {"command": "move", "parameters": {"direction": "depan"}}, {"command": "suction_cup", "parameters": {"action": "off"}}]}


127.0.0.1 - - [06/Jul/2024 18:51:31] "GET /api HTTP/1.1" 200 -


{"actions": [{"command": "err_msg", "parameters": {"msg": "tidak dapat membuat rencana aksi dengan kondisi terkini"}}]}


...

{"actions": [{"command": "move_to", "parameters": {"x": -147.04, "y": -353.61, "z": -50}}, {"command": "suction_cup", "parameters": {"action": "on"}}, {"command": "move_to", "parameters": {"x": 191.44, "y": -39.17, "z": -50}}, {"command": "move", "parameters": {"direction": "belakang"}}, {"command": "suction_cup", "parameters": {"action": "off"}}]}


127.0.0.1 - - [07/Jul/2024 01:56:18] "GET /api HTTP/1.1" 200 -


{"actions": [{"command": "err_msg", "parameters": {"msg": "tidak dapat membuat rencana aksi dengan kondisi terkini"}}]}


127.0.0.1 - - [07/Jul/2024 01:57:10] "GET /api HTTP/1.1" 200 -


{"actions": [{"command": "move_to", "parameters": {"x": 199.38, "y": 111.09, "z": -50}}, {"command": "suction_cup", "parameters": {"action": "on"}}, {"command": "move_to", "parameters": {"x": -147.04, "y": 253, "z": -50}}, {"command": "move", "parameters": {"direction": "depan"}}, {"command": "suction_cup", "parameters": {"action": "off"}}]}
